## Disponível online

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/carlos-adir/UnB-Courses/blob/main/mntf/trabalho3-pt2.ipynb)

Esse python notebook está disponível online no GitHub através do link:

* [GitHub/carlos-adir/UnB-Courses/mntf/trabalho3-pt2](https://github.com/carlos-adir/UnB-Courses/blob/main/mntf/trabalho3-pt2.ipynb)

In [1]:
import os
os.system("pip install numpy")
os.system("pip install tqdm")
import numpy as np
from tqdm import tqdm
from typing import List, Tuple, Callable, Optional
import warnings
warnings.filterwarnings("error")
np.set_printoptions(precision=3)

# Sistemas de Equações Lineares

### Teoria

Existem diversas formas de resolver sistemas lineares, mas eles se dividem em duas partes:

1. Métodos diretos

> $$[A] \cdot [X] = [B] \Rightarrow [X^{\star}] = [A]^{-1} \cdot [B]$$
> 
> Que o mais conhecido é através da elimininação gaussiana e suas variações.

2. Métodos iterativos

> A partir de um vetor $x_{0}$ inicial, calculamos sucessivamente novos vetor, que esperamos que se aproxime da solução exata $\left[X^{\star}\right]$.
> 
> $$\left[X_{k+1}\right] = \left[T\right] \cdot \left[X_{k}\right] + \left[C\right]$$
> 
> Com matrizes $\left[T\right]$ e $\left[C\right]$ que dependem do método utilizado.
> 
> Este método é muito parecido com o método do ponto fixo com uma variável, mas neste caso iteramos em vetores

Iremos nos atentar apenas nos métodos iterativos, e desta forma iremos nos aprofundar apenas neles. Para ele, seja $D$ a diagonal de $A$, $U$ a matriz triangular inferior de $A$ e $U$ a matriz triangular superior, então:

$$
[A] = \begin{bmatrix}
A_{11}&A_{12}&A_{13}\\
A_{21}&A_{22}&A_{23}\\
A_{31}&A_{32}&A_{33}
\end{bmatrix}
 = [D]+[L] +[U]
$$
$$
[D] = \begin{bmatrix}
A_{11} & 0 & 0 \\
0 & A_{22} & 0 \\
0 & 0 & A_{33}
\end{bmatrix}\ \  \ [L] = \begin{bmatrix}
0 & 0 & 0 \\
A_{21} & 0 & 0 \\
A_{31} & A_{32} & 0
\end{bmatrix}\
\ \ [U] = \begin{bmatrix}
0 & A_{12} & A_{13} \\
0 & 0 & A_{23} \\
0 & 0 & 0
\end{bmatrix}
$$

1. Método de Jacobi - [Link wikipedia](https://en.wikipedia.org/wiki/Jacobi_method)

> Temos 
>
> $$[A] \cdot [X] = [B]$$
> $$\left([D]+[L]+[U]\right) \cdot [X] = [B]$$
> $$[D] \cdot [X] = \left([L]+[U]\right) \cdot [X] + [B]$$
> $$[X] = \underbrace{[D]^{-1} \cdot \left([L]+[U]\right)}_{[T]} \cdot [X] + \underbrace{[D]^{-1} \cdot [B]}_{[C]}$$
> $$\begin{cases}[T] = [D]^{-1} \cdot \left([L]+[U]\right)\\ [C] = [D]^{-1} \cdot [B]\end{cases}$$
>
> Como a matriz $[D]$ é diagonal, então teremos que $[D]^{-1} = [D^{-1}] = \left[\frac{1}{A_{ii}}\right]$, o que é facil de se obter
>
> Na notação indicial temos
>
> $$[X_{k+1}]_{i} = \dfrac{1}{A_{ii}} \cdot \left(B_{i} - \sum_{\begin{smallmatrix}j=1\\ j\ne i\end{smallmatrix}}^{n} A_{ij} \cdot [X_{k}]_{j} \right)$$


In [2]:
def Jacobi(A: np.ndarray, B: np.ndarray, X0: np.ndarray, atol: float, verbose = False) -> Tuple[np.ndarray, int]:
    n = len(B)
    iteration = 0
    itermax = 200
    Xnew = np.copy(X0)
    while True:
        if verbose:
            print(f"X[{iteration}] = ", Xnew)
        for i in range(n):
            Xnew[i] = B[i]
            Xnew[i] -= sum(A[i,:i]*X0[:i])
            Xnew[i] -= sum(A[i,i+1:]*X0[i+1:])
            Xnew[i] /= A[i, i]
        error = np.max(np.abs(Xnew-X0))
        if verbose:
            print("    error = %.2e" % error)
        if error < atol:
            return Xnew, iteration
        X0 = np.copy(Xnew)
        iteration += 1
        if iteration > itermax:
            error_msg = f"Jacobi doesn't converge."
            raise ValueError(error_msg)

2. Método de Gauss-Seidel - [Link wikipedia](https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method)

> Temos 
>
> $$[A] \cdot [X] = [B]$$
> $$\left([D]+[L]+[U]\right) \cdot [X] = [B]$$
> $$\left([D]+[L]\right) \cdot [X] =   [U] \cdot [X] + [B]$$
> $$[X] = \underbrace{\left([D]+[L]\right)^{-1} \cdot [U]}_{[T]} \cdot [X] + \underbrace{\left([D]+[L]\right)^{-1} \cdot [B]}_{[C]}$$
> $$\begin{cases}[T] = \left([D]+[L]\right)^{-1} \cdot [U] \\ [C] = \left([D]+[L]\right)^{-1} \cdot [B]\end{cases}$$
>
> A inversa de $[D]+[L]$ é facil de conseguir pois a matriz é triangular e se pode obter $[M]$:
>
> $$\left([D]+[L]\right)^{-1} = \begin{bmatrix}A_{11} & 0 & \cdots & 0 \\ A_{21} & A_{22} & \cdots & 0 \\ \vdots & \vdots & \ddots & \vdots \\ A_{n1} & A_{n2} & \cdots & A_{nn}\end{bmatrix}^{-1} = \begin{bmatrix}M_{11} & 0 & \cdots & 0 \\ M_{21} & M_{22} & \cdots & 0 \\ \vdots & \vdots & \ddots & \vdots \\ M_{n1} & M_{n2} & \cdots & M_{nn}\end{bmatrix} = [M]$$
>
> Pode-se obter o mesmo resultado através da iteração utilizando a substituição progressiva:
>
> $$[X_{k+1}]_{i} = \dfrac{1}{A_{ii}} \cdot \left(B_{i} - \sum_{j=1}^{i-1} A_{ij} \cdot [X_{k+1}]_{j} -  \sum_{j=i+1}^{n} A_{ij} \cdot [X_{k}]_{j} \right)$$

In [3]:
def GaussSeidel(A: np.ndarray, B: np.ndarray, X0: np.ndarray, atol: float, verbose = False) -> Tuple[np.ndarray, int]:
    n = len(B)
    iteration = 0
    itermax = 200
    Xnew = np.copy(X0)
    while True:
        if verbose:
            print(f"X[{iteration}] = ", Xnew)
        for i in range(n):
            Xnew[i] = B[i]
            Xnew[i] -= sum(A[i,:i]*Xnew[:i])
            Xnew[i] -= sum(A[i,i+1:]*X0[i+1:])
            Xnew[i] /= A[i, i]
        error = np.max(np.abs(Xnew-X0))
        if verbose:
            print("    error = %.2e" % error)
        if error < atol:
            return Xnew, iteration
        X0 = np.copy(Xnew)
        iteration += 1
        if iteration > itermax:
            error_msg = f"Gauss Seidel doesn't converge."
            raise ValueError(error_msg)

3. Método SOR - [Link wikipedia](https://en.wikipedia.org/wiki/Successive_over-relaxation)

> Temos 
>
> $$[A] \cdot [X] = [B]$$
> $$\left([D]+[L]+[U]\right) \cdot [X] = [B]$$
> $$\omega \left([D]+[L]+[U]\right) \cdot [X] = \omega \cdot [B]$$
> $$\omega [L] \cdot [X] = \omega [B] -\left(\omega [U] + \omega[D]\right) \cdot [X]$$
> $$\left([D]+\omega [L] \right)\cdot [X] = \omega [B] - \left(\omega[U] + (\omega-1)[D]\right)\cdot [X]$$
> $$[X] = \underbrace{-\left([D]+\omega [L]\right)^{-1} \cdot \left(\omega[U]+(\omega-1)[D]\right)}_{[T]} \cdot [X] + \underbrace{\left([D]+\omega [L]\right)^{-1} \cdot [B]}_{[C]}$$
> $$\begin{cases}[T] = -\left([D]+\omega [L]\right)^{-1} \cdot \left(\omega[U]+(\omega-1)[D]\right) \\ [C] = \left([D]+\omega [L]\right)^{-1} \cdot [B]\end{cases}$$
>
> Pode-se obter o mesmo resultado através da iteração utilizando a substituição progressiva:
>
> $$[X_{k+1}]_{i} = (1-\omega) [X_{k}]_{i}+ \dfrac{\omega}{A_{ii}} \cdot \left(B_{i} - \sum_{j=1}^{i-1} A_{ij} \cdot [X_{k+1}]_{j} -  \sum_{j=i+1}^{n} A_{ij} \cdot [X_{k}]_{j} \right)$$

In [4]:
def SORMethod(A: np.ndarray, B: np.ndarray, X0: np.ndarray, omega: float, atol: float, verbose = False) -> Tuple[np.ndarray, int]:
    n = len(B)
    iteration = 0
    itermax = 200
    Xnew = np.copy(X0)
    while True:
        if verbose:
            print(f"X[{iteration}] = ", Xnew)
        for i in range(n):
            Xnew[i] = B[i]
            Xnew[i] -= sum(A[i,:i]*Xnew[:i])
            Xnew[i] -= sum(A[i,i+1:]*X0[i+1:])
            Xnew[i] *= omega/A[i, i]
            Xnew[i] += (1-omega)*X0[i]
        error = np.max(np.abs(Xnew-X0))
        if verbose:
            print("    error = %.2e" % error)
        if error < atol:
            return Xnew, iteration
        X0 = np.copy(Xnew)
        iteration += 1
        if iteration > itermax:
            error_msg = f"SOR Method doesn't converge."
            raise ValueError(error_msg)

4. Gradiente Conjulgado - [Link wikipedia](https://en.wikipedia.org/wiki/Conjugate_gradient_method)

> Este método é utilizado partindo da ideia que resolver o sistema linear $[A]\cdot [X] = [B]$ significa que queremos encontrar o valor de $[X]$ tal que minimize a função
>
> $$J([X]) = \dfrac{1}{2}[X]^{T} \cdot [A] \cdot [X] - [X]^{T} \cdot [B]$$
> 
> De tal forma que a derivada de $J$ em relação a $X$ significa:
>
> $$\dfrac{dJ}{dX} = A \cdot X - B $$
> 
> A ideia deste método consiste em dizer 

In [5]:
def GradienteConjugado(A: np.ndarray, B: np.ndarray, X0: np.ndarray, omega: float, atol: float, verbose=False) -> Tuple[np.ndarray, int]:
    n = len(B)
    iteration = 0
    itermax = 200
    r0 = B - A @ X0
    p0 = r0
    Xnew = np.copy(X0)
    while True:
        if verbose:
            print(f"X[{iteration}] = ", Xnew)
        alpha = np.inner(r0, r0)/(p0 @ A @ p0)
        Xnew = X0 + alpha * p0
        rnew = r0 - alpha * A @ p0
        error = np.max(np.abs(Xnew-X0))
        if verbose:
            print("    error = %.2e" % error)
        if error < atol:
            return Xnew, iteration
        beta = np.inner(rnew, rnew)/np.inner(r0, r0)
        p0 = rnew + beta * p0
        r0 = np.copy(rnew)
        iteration += 1
        if iteration > itermax:
            error_msg = f"Gradiente conjulgado doesn't converge."
            raise ValueError(error_msg)

### Exercicio 10

Use o método de Gauss-Seidel para resolver o seguinte sistema:

$$
\begin{bmatrix}
12 & -2 & 3 & -1 \\
1 & 6 & 20 & -4 \\
-2 & 15 & 6 & -3 \\
0 & -3 & 2 & 9
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3\\
x_4
\end{bmatrix}=
\begin{bmatrix}
0 \\
20\\
0\\
0
\end{bmatrix}
$$

A solução deste sistema é dada por

$$
X^{\star} =
\dfrac{1}{2653}
\begin{bmatrix}
-1092 \\ -1556 \\ 2940 \\ -1172
\end{bmatrix}
\approx 
\begin{bmatrix}
-0.4116  \\ -5865 \\ 1.1082 \\ -0.4418
\end{bmatrix}
$$

In [6]:
A = [[12, -2,  3, -1],
     [ 1,  6, 20, -4],
     [-2, 15,  6, -3],
     [ 0, -3,  2,  9]]
B = [0, 20, 0, 0]
A = np.array(A, dtype="float64")
B = np.array(B, dtype="float64")
Xstar = np.array([-1092, -1556, 2940, -1172], dtype="float64")/2653
print(f"norm L2 (A * Xstar - B) = {np.linalg.norm(A@Xstar-B):.2e}")

norm L2 (A * Xstar - B) = 1.33e-15


Podemos ver que o método diretamente não resolve e diverge:

In [7]:
try:
    X0 = np.random.uniform(-1, 1, B.shape)
    Xf, iter = GaussSeidel(A, B, X0, 1e-9, verbose=True)
except ValueError as e:
    print("Nao foi possivel encontrar a solucao do sistema.")
    print("Com a condicao iniciao X0 = ", X0)
    print("    Erro = ", e)

X[0] =  [-0.156  0.586 -0.344 -0.911]
    error = 9.71e+00
X[1] =  [  0.108   3.856 -10.059   3.521]
    error = 8.36e+01
X[2] =  [  3.451  38.635 -93.678  33.696]
    error = 7.10e+02
X[3] =  [  32.667  332.613 -803.796  289.492]
    error = 6.03e+03
X[4] =  [  280.509  2828.895 -6833.989  2461.629]
    error = 5.12e+04
X[5] =  [  2385.115  24026.862 -58041.303  20907.021]
    error = 4.35e+05
X[6] =  [  20257.055  204036.182 -492884.592  177541.97 ]
    error = 3.69e+06
X[7] =  [  172022.343  1732642.897 -4185495.477  1507657.738]
    error = 3.14e+07
X[8] =  [  1460785.83   14713295.776 -35542481.96   12802761.25 ]
    error = 2.66e+08
X[9] =  [ 1.240e+07  1.249e+08 -3.018e+08  1.087e+08]
    error = 2.26e+09
X[10] =  [ 1.053e+08  1.061e+09 -2.563e+09  9.232e+08]
    error = 1.92e+10
X[11] =  [ 8.945e+08  9.010e+09 -2.176e+10  7.840e+09]
    error = 1.63e+11
X[12] =  [ 7.596e+09  7.651e+10 -1.848e+11  6.657e+10]
    error = 1.38e+12
X[13] =  [ 6.450e+10  6.497e+11 -1.569e+12  5.653e

Para resolvermos, iremos fazer um pré-condicionamento e trocar as linhas $2$ e $3$ para obter:

$$
\begin{bmatrix}
12 & -2 & 3 & -1 \\
-2 & 15 & 6 & -3 \\
1 & 6 & 20 & -4 \\
0 & -3 & 2 & 9
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3\\
x_4
\end{bmatrix}=
\begin{bmatrix}
0 \\
0\\
20\\
0
\end{bmatrix}
$$

In [8]:
A = [[12, -2,  3, -1],
     [-2, 15,  6, -3],
     [ 1,  6, 20, -4],
     [ 0, -3,  2,  9]]
B = [0, 0, 20, 0]
A = np.array(A, dtype="float64")
B = np.array(B, dtype="float64")
X0 = np.random.uniform(-1, 1, B.shape)
Xf, iter = GaussSeidel(A, B, X0, 1e-9, verbose=True)
print("Final solution = ")
print("    ", Xf)
print("Exact solution = ")
print("    ", Xstar)

X[0] =  [-0.999  0.111 -0.749 -0.822]
    error = 1.53e+00
X[1] =  [ 0.137  0.153  0.783 -0.123]
    error = 5.15e-01
X[2] =  [-0.18  -0.362  1.093 -0.363]
    error = 1.97e-01
X[3] =  [-0.364 -0.558  1.113 -0.433]
    error = 4.36e-02
X[4] =  [-0.407 -0.586  1.11  -0.442]
    error = 4.48e-03
X[5] =  [-0.412 -0.587  1.108 -0.442]
    error = 4.94e-04
X[6] =  [-0.412 -0.587  1.108 -0.442]
    error = 1.40e-04
X[7] =  [-0.412 -0.587  1.108 -0.442]
    error = 2.52e-05
X[8] =  [-0.412 -0.587  1.108 -0.442]
    error = 1.75e-06
X[9] =  [-0.412 -0.587  1.108 -0.442]
    error = 3.85e-07
X[10] =  [-0.412 -0.587  1.108 -0.442]
    error = 9.54e-08
X[11] =  [-0.412 -0.587  1.108 -0.442]
    error = 1.34e-08
X[12] =  [-0.412 -0.587  1.108 -0.442]
    error = 6.49e-10
Final solution = 
     [-0.412 -0.587  1.108 -0.442]
Exact solution = 
     [-0.412 -0.587  1.108 -0.442]
